### Criação de um modelo matemático para prever a nota de redação do enem com base em seus componentes usando regressão linear multipla

### Analise Inicial<br>

In [151]:
import pandas as pd
import numpy as np


In [152]:
#Carregamento dos dados já separados em componentes

componentes = pd.read_csv("Componentes_Redacao_Enem_2021.csv")

In [153]:
#descrição da base de dados
print("\nDimensões:\n{0}\n".format(componentes.shape))
print("\nCampos:\n{0}\n".format(list(componentes.keys())))
print("\nTipos dos dados:\n{0}\n".format(componentes.dtypes))


Dimensões:
(3389832, 7)


Campos:
['Unnamed: 0', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']


Tipos dos dados:
Unnamed: 0           int64
NU_NOTA_COMP1      float64
NU_NOTA_COMP2      float64
NU_NOTA_COMP3      float64
NU_NOTA_COMP4      float64
NU_NOTA_COMP5      float64
NU_NOTA_REDACAO    float64
dtype: object



#### Descrição dos componentes:

NU_NOTA_COMP1: Nota da competência 1 - Demonstrar domínio da modalidade escrita formal da Língua Portuguesa.<br>
<br>
NU_NOTA_COMP2: Nota da competência 2 - Compreender a proposta de redação e aplicar conceitos das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo em prosa.<br>
<br>
NU_NOTA_COMP3: Nota da competência 3 - Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista.<br>
<br>
NU_NOTA_COMP4: Nota da competência 4 - Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação.<br>
<br>
NU_NOTA_COMP5: Nota da competência 5 - Elaborar proposta de intervenção para o problema abordado, respeitando os direitos humanos.<br>
<br>
NU_NOTA_REDACAO: Nota da prova de redação

In [154]:
print(componentes.head(10))

   Unnamed: 0  NU_NOTA_COMP1  NU_NOTA_COMP2  NU_NOTA_COMP3  NU_NOTA_COMP4  \
0           0          140.0          120.0          120.0          180.0   
1           1          120.0          120.0          120.0          120.0   
2           2            NaN            NaN            NaN            NaN   
3           3          120.0          180.0          120.0          200.0   
4           4          120.0          140.0          160.0          180.0   
5           5            NaN            NaN            NaN            NaN   
6           6          100.0          120.0          100.0          120.0   
7           7            NaN            NaN            NaN            NaN   
8           8          120.0          120.0           80.0          100.0   
9           9           80.0          120.0           80.0           80.0   

   NU_NOTA_COMP5  NU_NOTA_REDACAO  
0          200.0            760.0  
1           80.0            560.0  
2            NaN              NaN  
3       

In [155]:
#valores nulos
print("\nValores Faltantes:\n{0}\n".format(componentes.isnull().sum()))


Valores Faltantes:
Unnamed: 0               0
NU_NOTA_COMP1      1011453
NU_NOTA_COMP2      1011453
NU_NOTA_COMP3      1011453
NU_NOTA_COMP4      1011453
NU_NOTA_COMP5      1011453
NU_NOTA_REDACAO    1011453
dtype: int64



Estão faltando dados de 1.011.453 redações. Como a falta dessas notas irá prejudicar o modelo, esses dados serão retirados. 
Embora esses dados representem cerca de 1/3 da base de dados, substituir esses valores por 0 ou pelos valores da média dos dados, não irá trazer benefícios ao modelo

In [156]:
#Remoção dos valores nulos
componentes = componentes.dropna()


In [157]:
#confirmação da remoção dos valores nulos
print("\nValores Faltantes:\n{0}\n".format(componentes.isnull().sum()))



Valores Faltantes:
Unnamed: 0         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP3      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_REDACAO    0
dtype: int64



In [158]:
# Verificação de correlação entre as variáveis
componentes.corr()

,Unnamed: 0,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
Unnamed: 0,1.000000,-0.075141,-0.115022,-0.099980,-0.110891,-0.135339,-0.125135
NU_NOTA_COMP1,-0.075141,1.000000,0.719232,0.805451,0.824612,0.611643,0.863942
NU_NOTA_COMP2,-0.115022,0.719232,1.000000,0.814748,0.777865,0.678650,0.894539
NU_NOTA_COMP3,-0.099980,0.805451,0.814748,1.000000,0.838510,0.694860,0.918302
NU_NOTA_COMP4,-0.110891,0.824612,0.777865,0.838510,1.000000,0.696472,0.915502
NU_NOTA_COMP5,-0.135339,0.611643,0.678650,0.694860,0.696472,1.000000,0.860290
NU_NOTA_REDACAO,-0.125135,0.863942,0.894539,0.918302,0.915502,0.860290,1.000000


É importante para criação do modelo que as variáveis não possuam correlação, para isso serão aceitas correlações < 0,9.

Avaliando a correlação entre as variáveis e a correlação com a nota do enem, que é nosso objetivo de previsão, serão utilizadas as componentes 2, 4 e 5 por terem menor correlação entre si e maior correlação com a nota da redação.

#### Separação da base de treino e teste 

In [159]:
from sklearn.model_selection import train_test_split

In [160]:
# Particiona a base de dados


X = componentes[['NU_NOTA_COMP2', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']]

y = componentes[['NU_NOTA_REDACAO']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

In [161]:
from sklearn.linear_model import LinearRegression

In [162]:

print("\nVariáveis independentes:\n{0}\n".format(X_train.head()))


Variáveis independentes:
        NU_NOTA_COMP2  NU_NOTA_COMP4  NU_NOTA_COMP5
713173            0.0            0.0            0.0
361491          120.0          120.0          140.0
956439          200.0          180.0          160.0
986035          120.0          140.0           40.0
616636          200.0          140.0          120.0



In [163]:
print("\nVariavel dependente:\n{0}\n".format(y_train.head()))


Variavel dependente:
        NU_NOTA_REDACAO
713173              0.0
361491            600.0
956439            840.0
986035            520.0
616636            700.0



In [164]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [165]:
print("\nIntercepto:\n{0}\n".format(modelo.intercept_))


Intercepto:
[34.63905109]



In [166]:
print("\nCoeficientes:\n{0}\n".format(modelo.coef_))


Coeficientes:
[[1.43629753 2.0171083  1.082881  ]]



In [167]:
print("Equação: NOTA REDAÇÃO = {:.2f} + {:.2f}*NU_NOTA_COMP2 + {:.2f}*NU_NOTA_COMP4 + {:.2f}*NU_NOTA_COMP5".format(modelo.intercept_[0], modelo.coef_[0][0], modelo.coef_[0][1], modelo.coef_[0][2]))

Equação: NOTA REDAÇÃO = 34.64 + 1.44*NU_NOTA_COMP2 + 2.02*NU_NOTA_COMP4 + 1.08*NU_NOTA_COMP5


In [168]:
y_prediction =  modelo.predict(X_test)


In [169]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [170]:
score=r2_score(y_test,y_prediction)
print("\nScore:\n{0}\n".format(score))
print("\nmean_squared_error:\n{0}\n".format(mean_squared_error(y_test, y_prediction)))
print("\nmean_absolute_error:\n{0}\n".format(mean_absolute_error(y_test, y_prediction)))


Score:
0.9767480484145006


mean_squared_error:
844.7870602529442


mean_absolute_error:
23.11463362695624



In [171]:
# Criando uma coluna com previsão

previsao = pd.DataFrame(X_test)
previsao['NOTA_REAL'] = y_test
previsao['NOTA_PREVISTA'] = y_prediction
    
print(previsao)

         NU_NOTA_COMP2  NU_NOTA_COMP4  NU_NOTA_COMP5  NOTA_REAL  NOTA_PREVISTA
1419965          120.0          120.0          160.0      620.0     622.308711
3055296          120.0          120.0          120.0      600.0     578.993471
2311912          120.0          120.0           80.0      560.0     535.678231
1150127          120.0          120.0           80.0      540.0     535.678231
3345609          180.0          120.0            0.0      540.0     535.225603
...                ...            ...            ...        ...            ...
741721           200.0          160.0          160.0      780.0     817.896845
1261477          120.0          120.0          140.0      640.0     600.651091
2675661          160.0          100.0          120.0      620.0     596.103206
1418674          120.0          100.0           20.0      460.0     430.363205
605861           120.0           80.0           20.0      400.0     390.021039

[237838 rows x 5 columns]


Com esse modelo é possível fazer a previsão da nota de redação do Enem contendo apenas 3 das 5 componentes utilizadas para o cálculo da nota. 
O modelo mostrou ter uma boa capacidade de previsão com um score de 0,98 e um erro médio de 23.11 pontos.